In [1]:
!pip install PyMuPDF transformers datasets nltk torch


In [2]:
import os
import fitz  # PyMuPDF
import re
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

nltk.download('punkt')

c:\Users\email\Desktop\DS_Science_Youth_Extension\2504_Science_Youth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\email\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def extract_text_from_pdfs(pdf_folder):
    """Extract text from all PDFs in a given folder."""
    all_text = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            with fitz.open(pdf_path) as doc:
                text = "\n".join([page.get_text("text") for page in doc])
                all_text.append(text)
    return "\n".join(all_text)


In [4]:
def clean_text(text):
    """Normalize and clean extracted text."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove excessive spaces
    text = re.sub(r'[^a-z0-9.,!?\-\s]', '', text)  # Remove special characters
    return text


In [5]:
def tokenize_text(text):
    """Tokenize the text into sentences."""
    return nltk.sent_tokenize(text)


In [6]:
def prepare_dataset(texts, labels):
    """Create a Hugging Face Dataset for fine-tuning."""
    dataset = Dataset.from_dict({"text": texts, "label": labels})
    return dataset


In [7]:
def fine_tune_bert(dataset):
    """Fine-tune BERT for classification."""
    model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust labels
    
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length")
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    
    training_args = TrainingArguments(output_dir="../results", num_train_epochs=3, per_device_train_batch_size=8)
    trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)
    
    trainer.train()
    


In [8]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\email\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
!pip install "accelerate>=0.26.0"

In [10]:
!python.exe -c "import accelerate; print(accelerate.__version__)"

1.5.2


In [11]:
if __name__ == "__main__":
    pdf_folder = "../data/Training_docs"  # Update your path
    raw_text = extract_text_from_pdfs(pdf_folder)
    clean_text_data = clean_text(raw_text)
    tokenized_sentences = tokenize_text(clean_text_data)
    
    labels = [0] * len(tokenized_sentences)  # Dummy labels (Update with real labels if available)
    dataset = prepare_dataset(tokenized_sentences, labels)
    
    fine_tune_bert(dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 13581/13581 [00:06<00:00, 2097.34 examples/s]


Step,Training Loss
500,0.006300
1000,0.000000


KeyboardInterrupt: 